In [1]:
import requests
import time
import json
from transformers import AutoTokenizer

# Load Qwen3 tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B")

/root/llms/Qwen3_GPUS_metrics/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import requests
response = requests.get("http://localhost:8000/v1/models")
response.json()['data'][0]['root']

'Qwen/Qwen3-4B'

In [ ]:
import os
import glob

def measure_vllm_response(file_path, vllm_url="http://localhost:8000/v1/chat/completions", 
                         max_input_tokens=None, max_output_tokens=1024, max_generation_time=7.0):
    """
    Send file content to vLLM chat endpoint and measure response metrics with streaming
    
    Args:
        file_path: Path to the input file
        vllm_url: vLLM endpoint URL
        max_input_tokens: Maximum tokens for input (for truncation), None for no limit
        max_output_tokens: Maximum tokens for output response
        max_generation_time: Maximum time in seconds for generation after prefill (default: 7.0)
    """
    # Read the file
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Truncate content if max_input_tokens is specified
    if max_input_tokens is not None:
        tokens = tokenizer.encode(content)
        if len(tokens) > max_input_tokens:
            truncated_tokens = tokens[:max_input_tokens]
            content = tokenizer.decode(truncated_tokens)
            print(f"Content truncated from {len(tokens)} to {max_input_tokens} tokens")
    
    # Get actual input token count
    input_tokens = len(tokenizer.encode(content))
    
    # Prepare chat request with streaming
    payload = {
        "model": "qwen3",
        "messages": [
            {"role": "user", "content": content}
        ],
        "max_tokens": max_output_tokens,
        "temperature": 0.7,
        "stream": True,
        "stream_options": {"include_usage": True},  # Request usage info in stream
        "chat_template_kwargs": {
                "enable_thinking": False
        }
    }
    
    headers = {
        "Content-Type": "application/json"
    }
    
    # Start timing
    start_time = time.time()
    ttft = None
    generation_start_time = None
    generation_stopped_early = False
    
    # Send request to vLLM with streaming
    response = requests.post(vllm_url, json=payload, headers=headers, stream=True)
    
    if response.status_code != 200:
        raise Exception(f"Request failed with status {response.status_code}: {response.text}")
    
    # Process streaming response
    full_response = ""
    output_tokens = 0
    actual_input_tokens = None
    actual_output_tokens = None
    
    print("Response streaming:")
    print("-" * 50)
    
    try:
        for line in response.iter_lines():
            if line:
                line = line.decode('utf-8')
                if line.startswith('data: '):
                    data_str = line[6:]  # Remove 'data: ' prefix
                    if data_str.strip() == '[DONE]':
                        break
                    
                    try:
                        data = json.loads(data_str)
                        
                        # Check for usage information (comes in final event)
                        if 'usage' in data:
                            actual_input_tokens = data['usage']['prompt_tokens']
                            actual_output_tokens = data['usage']['completion_tokens']
                            print(f"\nUsage info received: {actual_input_tokens} input tokens, {actual_output_tokens} output tokens")
                        
                        if 'choices' in data and len(data['choices']) > 0:
                            choice = data['choices'][0]
                            if 'delta' in choice and 'content' in choice['delta']:
                                current_time = time.time()
                                
                                # Record TTFT (time to first token)
                                if ttft is None:
                                    ttft = current_time - start_time
                                    generation_start_time = current_time
                                
                                # Check if generation time exceeds max_generation_time
                                if generation_start_time is not None:
                                    generation_elapsed = current_time - generation_start_time
                                    if generation_elapsed > max_generation_time:
                                        print(f"\nGeneration stopped early after {generation_elapsed:.2f} seconds (max: {max_generation_time}s)")
                                        generation_stopped_early = True
                                        # Force close the response stream
                                        response.close()
                                        break
                                
                                content_chunk = choice['delta']['content']
                                full_response += content_chunk
                                
                                # Stream to console
                                print(content_chunk, end='', flush=True)
                                
                    except json.JSONDecodeError:
                        continue
                
                # Check timeout after each line as well
                if generation_start_time is not None:
                    generation_elapsed = time.time() - generation_start_time
                    if generation_elapsed > max_generation_time:
                        print(f"\nGeneration stopped early after {generation_elapsed:.2f} seconds (max: {max_generation_time}s)")
                        generation_stopped_early = True
                        response.close()
                        break
    
    except Exception as e:
        # If there's any error during streaming, close the response
        response.close()
        if not generation_stopped_early:
            raise e
    
    print()  # New line after streaming
    print("-" * 50)
    
    end_time = time.time()
    total_time = end_time - start_time
    
    # If no TTFT was recorded (no tokens received), set it to total time
    if ttft is None:
        ttft = total_time
    
    # Use actual token counts from server if available, otherwise fall back to tokenizer estimate
    if actual_input_tokens is not None and actual_output_tokens is not None and not generation_stopped_early:
        input_tokens = actual_input_tokens
        output_tokens = actual_output_tokens
        print(f"Using server-reported token counts: {input_tokens} input, {output_tokens} output")
    else:
        # Fallback: estimate output tokens using tokenizer
        output_tokens = len(tokenizer.encode(full_response))
        print(f"Using tokenizer estimates: {input_tokens} input, {output_tokens} output")
        if generation_stopped_early:
            print("Note: Generation was stopped early due to time limit")
    
    # Calculate generation speed using actual token counts
    generation_time = max(total_time - ttft, 1e-9)
    gen_tokens_per_sec = output_tokens / generation_time
    
    # Total tokens
    total_tokens = input_tokens + output_tokens
    
    # Metrics dictionary
    metrics = {
        'ttft': ttft,
        'gen_tokens_per_sec': gen_tokens_per_sec,
        'total_tokens': total_tokens,
        'total_time': total_time,
        'input_tokens': input_tokens,
        'output_tokens': output_tokens,
        'generation_time': generation_time,
        'generation_stopped_early': generation_stopped_early
    }
    
    return full_response, metrics

def save_and_print_metrics(response, metrics, output_file):
    """
    Save response and metrics to file and print metrics
    """
    # Print metrics
    print(f"TTFT: {metrics['ttft']:.2f} seconds")
    print(f"Gen tokens/sec (post-TTFT): {metrics['gen_tokens_per_sec']:.2f}")
    # print(f"E2E tokens/sec (incl. prefill): {metrics['e2e_tokens_per_sec']:.2f}")
    print(f"Total tokens: {metrics['total_tokens']}")
    print(f"Input tokens: {metrics['input_tokens']}")
    print(f"Output tokens: {metrics['output_tokens']}")
    print(f"Total time: {metrics['total_time']:.2f} seconds")
    print(f"Generation time: {metrics['generation_time']:.2f} seconds")
    if metrics.get('generation_stopped_early', False):
        print("Generation was stopped early due to time limit")
    
    # Save to file
    with open(output_file, 'w', encoding='utf-8') as f:
        # Write metrics in the format similar to the examples
        f.write(f"ttft: {metrics['ttft']:.2f}\n")
        f.write(f"gen_tokens_per_second: {metrics['gen_tokens_per_sec']:.2f}\n")
        # f.write(f"e2e_tokens_per_second: {metrics['e2e_tokens_per_sec']:.2f}\n")
        f.write(f"total_tokens: {metrics['total_tokens']}\n")
        f.write(f"total_time: {metrics['total_time']:.2f}\n")
        f.write(f"input_tokens: {metrics['input_tokens']}\n")
        f.write(f"output_tokens: {metrics['output_tokens']}\n")
        f.write(f"generation_time: {metrics['generation_time']:.2f}\n")
        if metrics.get('generation_stopped_early', False):
            f.write("generation_stopped_early: true\n")
        f.write("\n")
        f.write(response)

# file_path = "tests/daily.txt"
# vllm_url = "http://localhost:8000/v1/chat/completions"
# max_input_tokens =  28000
# max_output_tokens = 1024

# measure_vllm_response(file_path, vllm_url, max_input_tokens, max_output_tokens)

In [ ]:
# Warmup request to prepare the model
print("Warming up the model with a test request...")
warmup_response, warmup_metrics = measure_vllm_response(
    file_path="tests/daily.txt",
    max_input_tokens=100,
    max_output_tokens=10
)
print(f"Warmup completed in {warmup_metrics['total_time']:.2f}s")
print("Model is ready for testing.\n")

file_path = "tests/book.txt"
# input_token_sizes = [1000, 5000, 10000, 15000 , 20000, 25000, 30000]
input_token_sizes = [30000]

import threading
import time
import os

def run_concurrent_test(file_path, max_input_tokens, max_output_tokens, request_id):
    """Run a single request for concurrent testing"""
    print(f"Starting request {request_id}")
    start = time.time()
    
    try:
        response, metrics = measure_vllm_response(
            file_path=file_path,
            max_input_tokens=max_input_tokens,
            max_output_tokens=max_output_tokens
        )
            
        end = time.time()
        print(f"Request {request_id} completed in {end - start:.2f}s")
        return metrics
        
    except Exception as e:
        print(f"Request {request_id} failed: {e}")
        return None

print("Testing different input token sizes with various concurrent requests:")

concurrent_counts = [2, 5] 

for token_size in input_token_sizes:
    print(f"\n" + "="*80)
    print(f"Testing with {token_size} input tokens")
    print("="*80)
    
    for concurrent_count in concurrent_counts:
        print(f"\n=== {token_size} tokens with {concurrent_count} concurrent requests ===")
        print(f"Waiting for all current requests to complete before starting this set...")
        
        threads = []
        start_time = time.time()
        results = []
        
        # Start concurrent requests
        for i in range(concurrent_count):
            thread = threading.Thread(
                target=lambda i=i: results.append(run_concurrent_test(file_path, token_size, 100, f"{token_size}_{concurrent_count}_{i+1}"))
            )
            threads.append(thread)
            thread.start()
        
        # Wait for ALL threads to complete before proceeding to next set
        print(f"Waiting for all {concurrent_count} threads to complete...")
        for i, thread in enumerate(threads):
            thread.join()
            print(f"Thread {i+1}/{concurrent_count} completed")
        
        end_time = time.time()
        total_concurrent_time = end_time - start_time
        
        # Filter out None results (failed requests only)
        valid_results = [r for r in results if r is not None]
        
        print(f"All {concurrent_count} requests completed in {total_concurrent_time:.2f}s")
        print(f"Valid results: {len(valid_results)}/{concurrent_count}")
        print(f"Average time per request: {total_concurrent_time/concurrent_count:.2f}s")
        
        # Save aggregated metrics
        # Get model name from vLLM API
        try:
            import requests
            response = requests.get("http://localhost:8000/v1/models")
            models_data = response.json()
            model_name = models_data['data'][0]['root'].split('/')[-1] if models_data['data'] else "unknown_model"
        except Exception as e:
            print(f"Failed to get model name from API: {e}")
            model_name = "Qwen3-unkown"  # fallback
            
        output_dir = f"speed_tests/T2_x2/{model_name}"
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, f"{token_size}_length_{concurrent_count}_parallel.txt")
        
        # Calculate average metrics
        if valid_results:
            avg_ttft = sum(r['ttft'] for r in valid_results) / len(valid_results)
            avg_gen_tokens_per_sec = sum(r['gen_tokens_per_sec'] for r in valid_results) / len(valid_results)
            avg_total_tokens = sum(r['total_tokens'] for r in valid_results) / len(valid_results)
            avg_total_time = sum(r['total_time'] for r in valid_results) / len(valid_results)
            avg_input_tokens = sum(r['input_tokens'] for r in valid_results) / len(valid_results)
            avg_output_tokens = sum(r['output_tokens'] for r in valid_results) / len(valid_results)
            
            # Create average metrics dictionary
            avg_metrics = {
                'ttft': avg_ttft,
                'gen_tokens_per_sec': avg_gen_tokens_per_sec,
                'total_tokens': avg_total_tokens,
                'total_time': avg_total_time,
                'input_tokens': avg_input_tokens,
                'output_tokens': avg_output_tokens
            }
            
            # Print metrics
            print(f"Average TTFT: {avg_metrics['ttft']:.2f} seconds")
            print(f"Average Tokens/sec: {avg_metrics['gen_tokens_per_sec']:.2f}")
            print(f"Average Total tokens: {avg_metrics['total_tokens']:.0f}")
            print(f"Average Total time: {avg_metrics['total_time']:.2f} seconds")
            
            # Save average metrics to file
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(f"ttft: {avg_metrics['ttft']:.2f}\n")
                f.write(f"gen_tokens_per_second: {avg_metrics['gen_tokens_per_sec']:.2f}\n")
                f.write(f"total_tokens: {avg_metrics['total_tokens']:.0f}\n")
                f.write(f"total_time: {avg_metrics['total_time']:.2f}\n")
                f.write(f"input_tokens: {avg_metrics['input_tokens']:.0f}\n")
                f.write(f"output_tokens: {avg_metrics['output_tokens']:.0f}\n")
                f.write(f"concurrent_requests: {concurrent_count}\n")
                f.write(f"valid_requests: {len(valid_results)}\n")
                f.write(f"total_concurrent_time: {total_concurrent_time:.2f}\n\n")
                f.write(f"Average metrics for {len(valid_results)}/{concurrent_count} concurrent requests of {token_size} tokens each\n\n")
                
                # Write individual request metrics for ALL valid results
                f.write("Individual request metrics:\n")
                for i, metrics in enumerate(valid_results):
                    f.write(f"\nRequest {i+1}:\n")
                    f.write(f"  ttft: {metrics['ttft']:.2f}\n")
                    f.write(f"  gen_tokens_per_second: {metrics['gen_tokens_per_sec']:.2f}\n")
                    f.write(f"  total_tokens: {metrics['total_tokens']}\n")
                    f.write(f"  total_time: {metrics['total_time']:.2f}\n")
                    f.write(f"  input_tokens: {metrics['input_tokens']}\n")
                    f.write(f"  output_tokens: {metrics['output_tokens']}\n")
        else:
            print("No valid results to save")
        
        # Add a pause between different concurrent count sets for the same token size
        print(f"Set [{token_size} tokens - {concurrent_count} parallel] completed. Proceeding to next set...")
        time.sleep(2)  # Brief pause to ensure system is ready for next set

Warming up the model with a test request...
Content truncated from 13961 to 100 tokens
Response streaming:
--------------------------------------------------
Кажется, вы говорите на рус
Usage info received: 111 input tokens, 10 output tokens

--------------------------------------------------
Using server-reported token counts: 111 input, 10 output
Warmup completed in 3.07s
Model is ready for testing.

Testing different input token sizes with various concurrent requests:

Testing with 30000 input tokens

=== 30000 tokens with 2 concurrent requests ===
Waiting for all current requests to complete before starting this set...
Starting request 30000_2_1
Starting request 30000_2_2
Waiting for all 2 threads to complete...


Token indices sequence length is longer than the specified maximum sequence length for this model (181863 > 131072). Running this sequence through the model will result in indexing errors


Content truncated from 181863 to 30000 tokens
Content truncated from 181863 to 30000 tokens
Response streaming:
--------------------------------------------------
Response streaming:
--------------------------------------------------
****ГлавГлаваа V V ( (продпродолжолжениеение)**)**



ООггннегегрривив по почувствчувствоваловал,, как как у его него д дрожрожьат уси лливапаетсяы.. Он Не неуж могели пр ониой забтира дальшелись, под
Generation stopped early after 7.03 seconds (max: 7.0s)

--------------------------------------------------
Using tokenizer estimates: 30000 input, 37 output
Note: Generation was stopped early due to time limit
Request 30000_2_2 completed in 95.50s
 саму Гремящую тропу? Он беспокойно распушил свою рыжую шерсть и почувствовал, как под боком щекочущий кошачий запах. Воздух был тяж
Generation stopped early after 7.16 seconds (max: 7.0s)

--------------------------------------------------
Using tokenizer estimates: 30000 input, 94 output
Note: Generation was stop